<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/handwriting_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, LeaveOneOut, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc , classification_report, confusion_matrix
from ipywidgets import interact, interactive, widgets, interact_manual, Dropdown

from catboost import CatBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from sklearn.svm import SVC

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import re

import warnings
warnings.filterwarnings("ignore")

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("taeefnajib/handwriting-data-to-detect-alzheimers-disease")

print("Path to dataset files:", path)

100%|██████████| 332k/332k [00:00<00:00, 916kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/taeefnajib/handwriting-data-to-detect-alzheimers-disease/versions/1


In [8]:
import os

# List the contents of the directory
directory_path = '/root/.cache/kagglehub/datasets/taeefnajib/handwriting-data-to-detect-alzheimers-disease/versions/1'
print(os.listdir(directory_path))

['data.csv']


In [10]:
data = pd.read_csv('/root/.cache/kagglehub/datasets/taeefnajib/handwriting-data-to-detect-alzheimers-disease/versions/1/data.csv')

print("Shape of the data:", data.shape)
data.head()

Shape of the data: (174, 452)


,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,P
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,P
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,P
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,P
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,P


In [11]:
data.info()

# Corrected dtypes to count numeric and categorical variables
numeric_vars = data.select_dtypes(include=['int64', 'float64']).columns
categorical_vars = data.select_dtypes(include=['object', 'category']).columns
print("===")
print(f"Number of numeric variables: {len(numeric_vars)}")
print(f"Number of categorical variables: {len(categorical_vars)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Columns: 452 entries, ID to class
dtypes: float64(300), int64(150), object(2)
memory usage: 614.6+ KB
===
Number of numeric variables: 450
Number of categorical variables: 2
